In [2]:
import pandas as pd
import numpy as np
Nagpur_gdf = pd.read_csv('NGDR_Nagpur.csv')
# Nagpur_gdf.describe().to_dict()

In [3]:
print(Nagpur_gdf.isnull().sum())  # Check for NaN values in each column

gid             0
objectid        0
sampleno        0
longitude       0
latitude        0
             ... 
te              0
tl              0
hg           1263
cd           1263
geometry        0
Length: 72, dtype: int64


In [4]:
# Nagpur_gdf.fillna(0.0, inplace=True)  # Remove rows with any NaN values

In [5]:
Nagpur_gdf.isnull().sum()

gid             0
objectid        0
sampleno        0
longitude       0
latitude        0
             ... 
te              0
tl              0
hg           1263
cd           1263
geometry        0
Length: 72, dtype: int64

In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import spacy
import plotly.graph_objects as go
from pykrige.ok import OrdinaryKriging
from scipy.interpolate import griddata
import base64
import os




Nagpur_gdf = pd.read_csv('NGDR_Nagpur.csv')


import textdistance
word_list = ["kriging","concentration","toposheet","interpolation","inverse distance weighted","idw","maximum","minimum","longitude","latitude","aluminum"]
def correct_typos(text, threshold=0.5):
    corrected_text = []
    words = text.split()
    for word in words:
        # Check if the word is misspelled
        suggestions = [w for w in word_list if textdistance.jaccard.normalized_similarity(w, word) >= threshold]
        if suggestions:
            corrected_word = max(suggestions, key=lambda x: textdistance.jaccard.normalized_similarity(x, word))
        else:
            corrected_word = word
        corrected_text.append(corrected_word)
    return ' '.join(corrected_text)


# In[5]:


import re

# Dictionary of chemical names and their formulas

chemicals = {
   'silicon dioxide': 'sio2',
    'aluminum oxide': 'al2o3',
    'iron(III) oxide': 'fe2o3',
    'titanium dioxide': 'tio2',
    'calcium oxide': 'cao',
    'magnesium oxide': 'mgo',
    'manganese(II) oxide': 'mno',
    'sodium oxide': 'na2o',
    'potassium oxide': 'k2o',
    'phosphorus pentoxide': 'p2o5',
    'loss on ignition': 'loi',
    'barium': 'ba',
    'gallium': 'ga',
    'scandium': 'sc',
    'vanadium': 'v',
    'thorium': 'th',
    'lead': 'pb',
    'nickel': 'ni',
    'cobalt': 'co',
    'rubidium': 'rb',
    'strontium': 'sr',
    'yttrium': 'y',
    'zirconium': 'zr',
    'niobium': 'nb',
    'chromium': 'cr',
    'copper': 'cu',
    'zinc': 'zn',
    'gold': 'au',
    'lithium': 'li',
    'cesium': 'cs',
    'arsenic': 'as_',
    'antimony': 'sb',
    'bismuth': 'bi',
    'selenium': 'se',
    'silver': 'ag',
    'beryllium': 'be',
    'germanium': 'ge',
    'molybdenum': 'mo',
    'tin': 'sn',
    'lanthanum': 'la',
    'cerium': 'ce',
    'praseodymium': 'pr',
    'neodymium': 'nd',
    'samarium': 'sm',
    'europium': 'eu',
    'terbium': 'tb',
    'gadolinium': 'gd',
    'dysprosium': 'dy',
    'holmium': 'ho',
    'erbium': 'er',
    'thulium': 'tm',
    'ytterbium': 'yb',
    'lutetium': 'lu',
    'hafnium': 'hf',
    'tantalum': 'ta',
    'tungsten': 'w',
    'uranium': 'u',
    'platinum': 'pt',
    'palladium': 'pd',
    'indium': 'in_',
    'fluorine': 'f',
    'tellurium': 'te',
    'thallium': 'tl',
    'mercury': 'hg',
    'cadmium': 'cd'
    # Add more chemicals as needed
}

# Function to extract chemical names and formulas from a sentence
def extract_chemicals(query):
    # Initialize an empty list to store the extracted elements
    elements = []

    # Convert the sentence to lowercase
    lower_sentence = query.lower()

    # Extract words from the sentence
    words = re.findall(r'\b\w+\b', lower_sentence)

    # Check if any chemical name or formula is present in the sentence
    for chemical, formula in chemicals.items():
        if chemical.lower() in lower_sentence or formula.lower() in words:
            elements.append(formula)

    # Return the list of extracted elements
    return elements


def extract_topo_no(str1):
    toposheet_numbers = []
    toposheet_pattern = r'\b\d+[a-zA-Z]+\d+\b'
    toposheet_numbers = re.findall(toposheet_pattern,str1)
    return toposheet_numbers



def create_kriging_map_from_query(query,df):
    t1 = extract_topo_no(query)
    e1 = extract_chemicals(query)
    
    for toposheet_no in t1:
        for element in e1:
            max_value = df[element].max()
            max_location = df[df[element] == max_value][['latitude', 'longitude']].iloc[0]
            max_lat, max_lon = max_location['latitude'], max_location['longitude']
            # Minimum value aur uske corresponding latitude, longitude find kar rahe hain
            min_value = df[element].min()
            min_location = df[df[element] == min_value][['latitude', 'longitude']].iloc[0]
            min_lat, min_lon = min_location['latitude'], min_location['longitude']
            return generate_kriging_map(df,element,max_value,max_location,max_lat,max_lon,min_value,min_location,min_lat,min_lon, toposheet_no,)



def generate_kriging_map(df, element, max_value, max_location, max_lat, max_lon, min_value, min_location, min_lat, min_lon, toposheet_number=None, variogram_model='spherical'):
    # Function body goes here
     # Filter the DataFrame by the specified toposheet number if provided
    if toposheet_number is not None:
        df = df[df['toposheet'] == toposheet_number]
    
    # Define grid resolution
    gridx = np.linspace(df['longitude'].min(), df['longitude'].max(), 100)
    gridy = np.linspace(df['latitude'].min(), df['latitude'].max(), 100)
    print("GRID X: \n", gridx)              
    print("GRID Y: \n", gridy)
    print("Longitude:", df['longitude'].values)
    print("Latitude:", df['latitude'].values)
    print("Element Values:", df[element].values)
    print("Variogram Model:", variogram_model)
    
    print("Data head:", df.head())
    print("Data description:", df.describe())
    print("Longitude range:", df['longitude'].min(), df['longitude'].max())
    print("Latitude range:", df['latitude'].min(), df['latitude'].max())
# Perform Ordinary Kriging
    OK = OrdinaryKriging(df['longitude'], df['latitude'], df[element], variogram_model=variogram_model)
    z_interp, ss = OK.execute('grid', gridx, gridy)
    print("Z Interpolated:", z_interp)
    print("Semi-variance:", ss)
    # Create the contour plot
    contour = go.Contour(
        z=z_interp.data,  # 2D array of the heatmap values
        x=gridx,  # X coordinates corresponding to 'z_interp'
        y=gridy,  # Y coordinates corresponding to 'z_interp'
        colorscale='YlOrRd',  # Match the colormap
        showscale=True  # Show the color scale bar
    )
    
    # Create the scatter plot with hover annotations
    scatter = go.Scatter(
        x=df['longitude'],
        y=df['latitude'],
        mode='markers',
        marker=dict(
            color=df[element],
            colorscale='YlOrRd',  # Match the colormap
            showscale=False,  # We already have a color scale from the contour
            line=dict(color='black', width=1)  # Black border around the scatter points
        ),
        text=df[element],  # Text for hover annotations
        hoverinfo='text'  # Show only the text on hover
    )
    
    # Create a figure and add the contour plot
    fig = go.Figure(data=[contour])
    
    # Add the scatter plot on top of the contour plot
    fig.add_trace(scatter)
    
    fig.add_annotation(
    text=f"<i>Maximum value (in ppm): <b>{max_value}</b> at longitude <b>{max_lat}</b> and latitude <b>{max_lat}</b></i>",
    xref="paper", yref="paper",
    x=0.5, y=1.2, showarrow=False,
    font=dict(size=14),
    align="center"
    )

    fig.add_annotation(
    text=f"<i>Minimum value (in ppm): <b>{min_value}</b> at longitude <b>{min_lat}</b> and latitude <b>{min_lat}</b></i>",
    xref="paper", yref="paper",
    x=0.5, y=1.1, showarrow=False,
    font=dict(size=14),
    align="center"
    )

    # Add a title to the map
    fig.update_layout(
    title=f"<b>Stream Sediment samples showing {element} Values(ppm)</b>",
    title_x=0.5,  # Center the title
    title_y=0.95,  # Add some space from the top
    margin=dict(t=120)  # Adjust margin to accommodate the annotations and title
    )
    fig.show()
    data  = fig.to_dict()
    layout = fig.to_dict()
#     data = fig.to_dict()
#     layout = fig.to_dict()
#     return (data,layout)


# In[103]:


def create_idw_map_from_query(query,df):
    t2 = extract_topo_no(query)
    e2 = extract_chemicals(query)
    threshold_percentile = 100   
    for toposheet_number in t2:
        for element in e2:
            max_value = df[element].max()
            max_location = df[df[element] == max_value][['latitude', 'longitude']].iloc[0]
            max_lat, max_lon = max_location['latitude'], max_location['longitude']
            # Minimum value aur uske corresponding latitude, longitude find kar rahe hain
            min_value = df[element].min()
            min_location = df[df[element] == min_value][['latitude', 'longitude']].iloc[0]
            min_lat, min_lon = min_location['latitude'], min_location['longitude']
            return generate_idw_map(df, element,max_value, max_location, max_lat, max_lon, min_value, min_location, min_lat, min_lon, toposheet_number, threshold_percentile) 


# In[181]:


import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
import io
import base64

# def generate_idw_map(df, element,max_value, max_location, max_lat, max_lon, min_value, min_location, min_lat, min_lon, toposheet_number, threshold_percentile):
#     # Filter the DataFrame by the specified toposheet number
#     gdf = df[df['toposheet'] == toposheet_number]

#     # Step 1: Determine Baseline
#     baseline = np.median(gdf[element])  # Using median as the baseline

#     # Step 2: Calculate Deviation
#     deviation = gdf[element] - baseline

#     # Step 3: Statistical Analysis
#     std_dev = np.std(deviation)
#     percentile_value = np.percentile(deviation, threshold_percentile)

#     # Step 4: Define Anomaly Threshold
#     anomaly_threshold = percentile_value  # Setting the specified percentile as the threshold

#     # Step 5: Identify Anomalies
#     anomalies = gdf[deviation > anomaly_threshold]

#     # Step 6: Anomaly Map
#     # Create grid coordinates for interpolation
#     grid_x, grid_y = np.mgrid[min(gdf['longitude']):max(gdf['longitude']):100j, min(gdf['latitude']):max(gdf['latitude']):100j]

#     # Interpolate using IDW
#     grid_z = griddata((gdf['longitude'], gdf['latitude']), deviation, (grid_x, grid_y), method='cubic')

#     # Plotting the IDW map
#     plt.figure(figsize=(10, 8))
#     plt.imshow(grid_z.T, extent=(min(gdf['longitude']), max(gdf['longitude']), min(gdf['latitude']), max(gdf['latitude'])), origin='lower')
#     plt.scatter(anomalies['longitude'], anomalies['latitude'], c='red')  # Anomalies marked in red
#     plt.colorbar(label='Deviation from Baseline')
#     plt.title(f'Geochemical IDW Map for {element} (Toposheet {toposheet_number})\n\nMaximum value (in ppm): {max_value} at longitude {max_lat} and latitude {max_lat}\nMinimum value (in ppm): {min_value} at longitude {min_lat} and latitude {min_lat}')

#     plt.xlabel('Longitude')
#     plt.ylabel('Latitude')
#     plt.show()

import plotly.graph_objs as go
import numpy as np
from scipy.interpolate import griddata


def generate_idw_map(df, element, max_value, max_location, max_lat, max_lon, min_value, min_location, min_lat, min_lon, toposheet_number, threshold_percentile):
    # Filter the DataFrame by the specified toposheet number
    gdf = df[df['toposheet'] == toposheet_number]

    # Determine Baseline
    baseline = np.median(gdf[element])

    # Calculate Deviation
    deviation = gdf[element] - baseline

    # Statistical Analysis
    std_dev = np.std(deviation)
    percentile_value = np.percentile(deviation, threshold_percentile)

    # Define Anomaly Threshold
    anomaly_threshold = percentile_value

    # Identify Anomalies
    anomalies = gdf[deviation > anomaly_threshold]

    # Create grid coordinates for interpolation
    grid_x, grid_y = np.mgrid[min(gdf['longitude']):max(gdf['longitude']):100j, min(gdf['latitude']):max(gdf['latitude']):100j]

    # Interpolate using IDW
    grid_z = griddata((gdf['longitude'], gdf['latitude']), deviation, (grid_x, grid_y), method='cubic')

    # Create the contour plot
    contour = go.Contour(
        z=grid_z.T,
        x=np.linspace(min(gdf['longitude']), max(gdf['longitude']), 100),
        y=np.linspace(min(gdf['latitude']), max(gdf['latitude']), 100),
        colorscale='Viridis',
        colorbar=dict(title='Deviation from Baseline')
    )

    # Create the scatter plot for anomalies
    scatter = go.Scatter(
        x=anomalies['longitude'],
        y=anomalies['latitude'],
        mode='markers',
        marker=dict(color='red', size=5),
        name='Anomalies'
    )

    # Define the layout
    layout = go.Layout(
        title=f'Geochemical IDW Map for {element} (Toposheet {toposheet_number})<br>'
              f'Maximum value (in ppm): {max_value} at longitude {max_lat} and latitude {max_lat}<br>'
              f'Minimum value (in ppm): {min_value} at longitude {min_lat} and latitude {min_lat}',
        xaxis=dict(title='Longitude'),
        yaxis=dict(title='Latitude')
    )
    
    # Create the figure and plot it
    fig = go.Figure(data=[contour, scatter], layout=layout)
    
#     ranges = [
#     (0, 50),
#     (50, 100),
#     (100, 150),
#     (150, 200),
#     (200, 250)
#     ]
#     colors = ['#ffffcc', '#c7e9b4', '#7fcdbb', '#41b6c4', '#2c7fb8']
    
#     # Calculate the number of ranges to determine the spacing
#     num_ranges = len(ranges)
#     height_per_rect = 0.05  # Adjust the height of each rectangle here
#     total_height = num_ranges * height_per_rect

#     # Update the layout to make space for the legend
#     fig.update_layout(
#         margin=dict(b=150)  # Increase bottom margin to make space for the legend
#     )

#     # Add rectangles and annotations for the legend
#     for i, (lower, upper) in enumerate(ranges):
#         fig.add_shape(
#             type="rect",
#             xref="paper", yref="paper",
#             x0=0.1, y0=(-0.2) - (i + 1) * height_per_rect, x1=0.2, y1=(-0.2) - i * height_per_rect,
#             fillcolor=colors[i],
#             line=dict(width=0)
#         )
#         fig.add_annotation(
#             x=0.25, y=(-0.25) - (i + 0.5) * height_per_rect,
#             text=f"{lower}-{upper}",
#             showarrow=False,
#             xref="paper", yref="paper"
#         )


    fig.show()






def find_max_values(query, df):
    toposheet_numbers = extract_topo_no(query)
    elements = extract_chemicals(query)
    for toposheet_number in toposheet_numbers:
        for element in elements:
            max_value = df[element].max()
            max_location = df[df[element] == max_value][['latitude', 'longitude']].iloc[0]
            max_lat, max_lon = max_location['latitude'], max_location['longitude']
            max_value_result = f"For the toposheet {toposheet_number}, the element {element} has maximum PPM value {max_value} at latitude {max_lat} and longitude {max_lon}."
            return max_value_result


# In[183]:


def find_min_values(query, df):
#     print("[INFO:] Finding the min values")
    toposheet_numbers = extract_topo_no(query)
    elements = extract_chemicals(query)
    for toposheet_number in toposheet_numbers:
        for element in elements:
            min_value = df[element].min()
            min_location = df[df[element] == min_value][['latitude', 'longitude']].iloc[0]
            min_lat, min_lon = min_location['latitude'], min_location['longitude']
    
            # Results ko sentence mein display kar rahe hain
            min_value_result = f"For the toposheet {toposheet_number}, the element {element} has minimum PPM value {min_value} at latitude {min_lat} and longitude {min_lon}."
            return min_value_result


# In[184]:


def find_both_min_max(query, df):
    toposheet_numbers = extract_topo_no(query)
    elements = extract_chemicals(query)
    for toposheet_number in toposheet_numbers:
        for element in elements:
            max_value = df[element].max()
            max_location = df[df[element] == max_value][['latitude', 'longitude']].iloc[0]
            max_lat, max_lon = max_location['latitude'], max_location['longitude']
            # Minimum value aur uske corresponding latitude, longitude find kar rahe hain
            min_value = df[element].min()
            min_location = df[df[element] == min_value][['latitude', 'longitude']].iloc[0]
            min_lat, min_lon = min_location['latitude'], min_location['longitude']

            # Results ko sentence mein display kar rahe hain
            min_max_value_result = f"For the toposheet {toposheet_number}, the element {element} has maximum PPM value {max_value} at latitude {max_lat} and longitude {max_lon}, and has minimum PPM value {min_value} at latitude {min_lat} and longitude {min_lon}."
            return min_max_value_result


# In[185]:


def split_query_smartly(query):
    element_names = ['silicon dioxide', 'aluminum oxide', 'iron(III) oxide', 'titanium dioxide', 'calcium oxide', 'magnesium oxide', 'manganese(II) oxide', 'sodium oxide', 'potassium oxide', 'phosphorus pentoxide', 'loss on ignition', 'barium', 'gallium', 'scandium', 'vanadium', 'thorium', 'lead', 'nickel', 'cobalt', 'rubidium', 'strontium', 'yttrium', 'zirconium', 'niobium', 'chromium', 'copper', 'zinc', 'gold', 'lithium', 'cesium', 'arsenic', 'antimony', 'bismuth', 'selenium', 'silver', 'beryllium', 'germanium', 'molybdenum', 'tin', 'lanthanum', 'cerium', 'praseodymium', 'neodymium', 'samarium', 'europium', 'terbium', 'gadolinium', 'dysprosium', 'holmium', 'erbium', 'thulium', 'ytterbium', 'lutetium', 'hafnium', 'tantalum', 'tungsten', 'uranium', 'platinum', 'palladium', 'indium', 'fluorine', 'tellurium', 'thallium', 'mercury', 'cadmium']
    # Load spaCy English model
    nlp = spacy.load("en_core_web_sm")
    
    # Define split words and phrases
    split_words = ["maximum and minimum", "longitude and latitude"]
    additional_split_words = ["also", "display", "create", "produce", "tell", "describe","show"]
    
    # Process the query using spaCy
    doc = nlp(query)
    
    # Initialize variables
    subqueries = []
    subquery = ""
    split_flag = False  # Flag to determine if splitting is allowed
    
    # Iterate over tokens in the query
    for token in doc:
        # Check if token is a split word or phrase
        if any(split_word in token.text.lower() for split_word in split_words):
            split_flag = False
        elif token.text.strip() == "." or token.text.strip() == ". ":
            # Split when encountering a full stop or full stop followed by a space
            if subquery.strip():
                subqueries.append(subquery.strip())
            subquery = ""
            split_flag = False
        elif token.text.strip() in additional_split_words:
            # Split when encountering additional split words
            if subquery.strip():
                subqueries.append(subquery.strip())
            subquery = ""
            split_flag = False
        elif token.text.strip() == "and" and split_flag:
            # Do not split when "and" is encountered between elements
            subquery += token.text_with_ws
        elif token.text.strip() == "and" and subquery.lower().strip() in element_names:
            # Do not split when "and" is encountered after an element name
            subquery += token.text_with_ws
        elif token.text.strip().lower() not in additional_split_words:
            # Append token to subquery if it's not in the additional split words list
            subquery += token.text_with_ws
            split_flag = True  # Set flag to allow splitting after this token
    
    # Append the final subquery if it's not empty and contains more than one word
    if subquery.strip() and len(subquery.split()) > 1:
        subqueries.append(subquery.strip())
    
    return subqueries



def process_subqueries(subqueries):
#     combined_output = []  # Initialize a list to store combined output
    df = Nagpur_gdf
    try: 
        for subquery in subqueries:
            print(f"Processing subquery: {subquery}")
            if ('maximum' in subquery.lower() and 'minimum' in subquery.lower()) or ('max' in subquery.lower() and 'min' in subquery.lower()):
                print("Calling the function for min and max")
                output = find_both_min_max(subquery, df=Nagpur_gdf) 
            elif ('maximum' in subquery.lower()) or ('max' in subquery.lower()):
                print('Calling Max Fun')
                output = find_max_values(subquery,df)
            elif ('minimum' in subquery.lower()) or ('min' in subquery.lower()):
                print('Calling Min Fun')
                output = find_min_values(subquery,df)      
                
            # Check if the subquery mentions 'idw interpolation map'
            elif 'idw' in subquery.lower() or 'inverse distance weighted map' in subquery.lower():
                print("Calling the IDW function")
                output = create_idw_map_from_query(subquery, df)           
                
            elif 'idw' in subquery.lower() and "kriging" in subquery.lower():
                print("Calling both kriging and IDW function")
                output_idw = create_idw_map_from_query(subquery, df)
                output_kriging = create_kriging_map_from_query(subquery, df)
                output = (output_idw, output_kriging)  # Store both outputs in a tuple
                
            elif 'kriging' in subquery.lower():
                print("Calling the kriging function")
                output = create_kriging_map_from_query(subquery, df)
            else:
                apology_message = "I'm only able to provide information related to Nagpur data. Please enter a valid query about Nagpur toposheet data. Thank you for your understanding and patience."
                print(apology_message)
        return output 
    except Exception as e:
        print("Error:", e)
        return "There was problem processing your query, please try again and make sure the query is valid on Nagpur toposheet data. Thank you for your understanding and patience."
# Example usage:
# SubQueries = [
#     'the maximum and minimum longitude and latitude values for 55K14 for gold',
#     'a kriging map for the gold for 55K14'
# ]

# combined_output = process_subqueries(SubQueries)
# print("Combined Output:", combined_output)


# In[200]:


def generate_geochemistry_response(query):
    corrected_sentence = correct_typos(query)
    Subqueries = split_query_smartly(corrected_sentence)
    response = process_subqueries(Subqueries)
    if response == None:
        response = "Sorry, I am unable to respond to this query. I am currently equipped to provide information on Nagpur Geochemistry Toposheet data and can handle one query at a time. Thank you for your understanding."
    data_type = "text"
    if type(response) == dict:
        data_type = "idw_map"
    elif type(response) == tuple:
        data_type = "kriging_map"
    return response, data_type



In [12]:
import pykrige
generate_geochemistry_response("create a kriging map of ag in 55K14 ")

Processing subquery: a kriging map of ag in 55K14
Calling the kriging function
GRID X: 
 [78.75965    78.76199374 78.76433747 78.76668121 78.76902495 78.77136869
 78.77371242 78.77605616 78.7783999  78.78074364 78.78308737 78.78543111
 78.78777485 78.79011859 78.79246232 78.79480606 78.7971498  78.79949354
 78.80183727 78.80418101 78.80652475 78.80886848 78.81121222 78.81355596
 78.8158997  78.81824343 78.82058717 78.82293091 78.82527465 78.82761838
 78.82996212 78.83230586 78.8346496  78.83699333 78.83933707 78.84168081
 78.84402455 78.84636828 78.84871202 78.85105576 78.85339949 78.85574323
 78.85808697 78.86043071 78.86277444 78.86511818 78.86746192 78.86980566
 78.87214939 78.87449313 78.87683687 78.87918061 78.88152434 78.88386808
 78.88621182 78.88855556 78.89089929 78.89324303 78.89558677 78.89793051
 78.90027424 78.90261798 78.90496172 78.90730545 78.90964919 78.91199293
 78.91433667 78.9166804  78.91902414 78.92136788 78.92371162 78.92605535
 78.92839909 78.93074283 78.9330865

('There was problem processing your query, please try again and make sure the query is valid on Nagpur toposheet data. Thank you for your understanding and patience.',
 'text')

In [8]:
generate_geochemistry_response("create a kriging map of cu in 55K14 ")

Processing subquery: a kriging map of cu in 55K14
Calling the kriging function
GRID X: 
 [78.75965    78.76199374 78.76433747 78.76668121 78.76902495 78.77136869
 78.77371242 78.77605616 78.7783999  78.78074364 78.78308737 78.78543111
 78.78777485 78.79011859 78.79246232 78.79480606 78.7971498  78.79949354
 78.80183727 78.80418101 78.80652475 78.80886848 78.81121222 78.81355596
 78.8158997  78.81824343 78.82058717 78.82293091 78.82527465 78.82761838
 78.82996212 78.83230586 78.8346496  78.83699333 78.83933707 78.84168081
 78.84402455 78.84636828 78.84871202 78.85105576 78.85339949 78.85574323
 78.85808697 78.86043071 78.86277444 78.86511818 78.86746192 78.86980566
 78.87214939 78.87449313 78.87683687 78.87918061 78.88152434 78.88386808
 78.88621182 78.88855556 78.89089929 78.89324303 78.89558677 78.89793051
 78.90027424 78.90261798 78.90496172 78.90730545 78.90964919 78.91199293
 78.91433667 78.9166804  78.91902414 78.92136788 78.92371162 78.92605535
 78.92839909 78.93074283 78.9330865

('There was problem processing your query, please try again and make sure the query is valid on Nagpur toposheet data. Thank you for your understanding and patience.',
 'text')

In [9]:
# generate_geochemistry_response("create a idw map of cu in 55P10")
# toposheet_list = Nagpur_gdf["toposheet"]
# list(toposheet_list)

In [10]:
element_list = list(chemicals.keys())
toposheet_list = list(Nagpur_gdf["toposheet"])
for element in element_list:
    for toposheet in toposheet_list:
        generate_geochemistry_response(f'create a kriging map for silver in toposheet no {toposheet}')
#         generate_geochemistry_response(f'create a idw map for {element} in toposheet no {toposheet}')

Processing subquery: a kriging map for silver in toposheet no 55K14
Calling the kriging function
GRID X: 
 [78.75965    78.76199374 78.76433747 78.76668121 78.76902495 78.77136869
 78.77371242 78.77605616 78.7783999  78.78074364 78.78308737 78.78543111
 78.78777485 78.79011859 78.79246232 78.79480606 78.7971498  78.79949354
 78.80183727 78.80418101 78.80652475 78.80886848 78.81121222 78.81355596
 78.8158997  78.81824343 78.82058717 78.82293091 78.82527465 78.82761838
 78.82996212 78.83230586 78.8346496  78.83699333 78.83933707 78.84168081
 78.84402455 78.84636828 78.84871202 78.85105576 78.85339949 78.85574323
 78.85808697 78.86043071 78.86277444 78.86511818 78.86746192 78.86980566
 78.87214939 78.87449313 78.87683687 78.87918061 78.88152434 78.88386808
 78.88621182 78.88855556 78.89089929 78.89324303 78.89558677 78.89793051
 78.90027424 78.90261798 78.90496172 78.90730545 78.90964919 78.91199293
 78.91433667 78.9166804  78.91902414 78.92136788 78.92371162 78.92605535
 78.92839909 78.9

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import spacy
import plotly.graph_objects as go
import pykrige
from pykrige.ok import OrdinaryKriging
from scipy.interpolate import griddata
import base64
import os
import io



# Get the directory of the current Python file
# script_dir = os.path.dirname(__file__)

# Construct the relative path to the CSV file (assuming it's in the same directory)
# csv_file_path = os.path.join(script_dir, 'NGDR_Nagpur.csv')
Nagpur_gdf = pd.read_csv('NGDR_Nagpur.csv')
# Nagpur_gdf.fillna(0.0, inplace=True)

import textdistance
word_list = ["kriging","concentration","toposheet","interpolation","inverse distance weighted","idw","maximum","minimum","longitude","latitude","aluminum"]
def correct_typos(text, threshold=0.5):
    corrected_text = []
    words = text.split()
    for word in words:
        # Check if the word is misspelled
        suggestions = [w for w in word_list if textdistance.jaccard.normalized_similarity(w, word) >= threshold]
        if suggestions:
            corrected_word = max(suggestions, key=lambda x: textdistance.jaccard.normalized_similarity(x, word))
        else:
            corrected_word = word
        corrected_text.append(corrected_word)
    return ' '.join(corrected_text)


# In[5]:


import re

# Dictionary of chemical names and their formulas
chemicals = {
   'silicon dioxide': 'sio2',
    'aluminum oxide': 'al2o3',
    'iron(III) oxide': 'fe2o3',
    'titanium dioxide': 'tio2',
    'calcium oxide': 'cao',
    'magnesium oxide': 'mgo',
    'manganese(II) oxide': 'mno',
    'sodium oxide': 'na2o',
    'potassium oxide': 'k2o',
    'phosphorus pentoxide': 'p2o5',
    'loss on ignition': 'loi',
    'barium': 'ba',
    'gallium': 'ga',
    'scandium': 'sc',
    'vanadium': 'v',
    'thorium': 'th',
    'lead': 'pb',
    'nickel': 'ni',
    'cobalt': 'co',
    'rubidium': 'rb',
    'strontium': 'sr',
    'yttrium': 'y',
    'zirconium': 'zr',
    'niobium': 'nb',
    'chromium': 'cr',
    'copper': 'cu',
    'zinc': 'zn',
    'gold': 'au',
    'lithium': 'li',
    'cesium': 'cs',
    'arsenic': 'as_',
    'antimony': 'sb',
    'bismuth': 'bi',
    'selenium': 'se',
    'silver': 'ag',
    'beryllium': 'be',
    'germanium': 'ge',
    'molybdenum': 'mo',
    'tin': 'sn',
    'lanthanum': 'la',
    'cerium': 'ce',
    'praseodymium': 'pr',
    'neodymium': 'nd',
    'samarium': 'sm',
    'europium': 'eu',
    'terbium': 'tb',
    'gadolinium': 'gd',
    'dysprosium': 'dy',
    'holmium': 'ho',
    'erbium': 'er',
    'thulium': 'tm',
    'ytterbium': 'yb',
    'lutetium': 'lu',
    'hafnium': 'hf',
    'tantalum': 'ta',
    'tungsten': 'w',
    'uranium': 'u',
    'platinum': 'pt',
    'palladium': 'pd',
    'indium': 'in_',
    'fluorine': 'f',
    'tellurium': 'te',
    'thallium': 'tl',
    'mercury': 'hg',
    'cadmium': 'cd'
    # Add more chemicals as needed
}

# Function to extract chemical names and formulas from a sentence
def extract_chemicals(query):
    # Initialize an empty list to store the extracted elements
    elements = []

    # Convert the sentence to lowercase
    lower_sentence = query.lower()

    # Extract words from the sentence
    words = re.findall(r'\b\w+\b', lower_sentence)

    # Check if any chemical name or formula is present in the sentence
    for chemical, formula in chemicals.items():
        if chemical.lower() in lower_sentence or formula.lower() in words:
            elements.append(formula)

    # Return the list of extracted elements
    return elements


def extract_topo_no(str1):
    toposheet_numbers = []
    toposheet_pattern = r'\b\d+[a-zA-Z]+\d+\b'
    toposheet_numbers = re.findall(toposheet_pattern,str1)
    return toposheet_numbers



def create_kriging_map_from_query(query,df):
    t1 = extract_topo_no(query)
    e1 = extract_chemicals(query)
    
    for toposheet_no in t1:
        for element in e1:
            max_value = df[element].max()
            max_location = df[df[element] == max_value][['latitude', 'longitude']].iloc[0]
            max_lat, max_lon = max_location['latitude'], max_location['longitude']
            # Minimum value aur uske corresponding latitude, longitude find kar rahe hain
            min_value = df[element].min()
            min_location = df[df[element] == min_value][['latitude', 'longitude']].iloc[0]
            min_lat, min_lon = min_location['latitude'], min_location['longitude']
            return generate_kriging_map(df,element, toposheet_no)
            # return generate_kriging_map(df,element,max_value,max_location,max_lat,max_lon,min_value,min_location,min_lat,min_lon, toposheet_no,)


# OLD KRIGING CODE
# def generate_kriging_map(df, element, max_value, max_location, max_lat, max_lon, min_value, min_location, min_lat, min_lon, toposheet_number=None, variogram_model='spherical'):
#     # Function body goes here
#      # Filter the DataFrame by the specified toposheet number if provided
#     if toposheet_number is not None:
#         df = df[df['toposheet'] == toposheet_number]
    
#     # Define grid resolution
#     gridx = np.linspace(df['longitude'].min(), df['longitude'].max(), 100)
#     gridy = np.linspace(df['latitude'].min(), df['latitude'].max(), 100)
    
#     # Perform Ordinary Kriging
#     OK = OrdinaryKriging(df['longitude'], df['latitude'], df[element], variogram_model=variogram_model)
#     z_interp, ss = OK.execute('grid', gridx, gridy)

#     # Fill NaN values with the minimum value
#     # fill_value = np.nanmin(z_interp.data) - 1
#     # z_filled = np.nan_to_num(z_interp.data, nan=fill_value)
    
#     # Create the contour plot
#     contour = go.Contour(
#         z=z_interp.data,  # 2D array of the heatmap values
#         x=gridx,  # X coordinates corresponding to 'z_interp'
#         y=gridy,  # Y coordinates corresponding to 'z_interp'
#         # contours=dict(
#         # start=np.nanmin(z_interp.data),
#         # end=np.nanmax(z_interp.data),
#         # size=(np.nanmax(z_interp.data) - np.nanmin(z_interp.data)) / 10,
#         # ),
#         colorscale='YlOrRd',  # Match the colormap
#         showscale=True  # Show the color scale bar
#     )
    
#     # Create the scatter plot with hover annotations
#     scatter = go.Scatter(
#         x=df['longitude'],
#         y=df['latitude'],
#         mode='markers',
#         marker=dict(
#             color=df[element],
#             colorscale='YlOrRd',  # Match the colormap
#             showscale=False,  # We already have a color scale from the contour
#             line=dict(color='black', width=1)  # Black border around the scatter points
#         ),
#         text=df[element],  # Text for hover annotations
#         hoverinfo='text'  # Show only the text on hover
#     )
    
#     # Create a figure and add the contour plot
#     fig = go.Figure(data=[contour])
    
#     # Add the scatter plot on top of the contour plot
#     # fig.add_trace(scatter)
    
#     fig.add_annotation(
#     text=f"<i>Maximum value (in ppm): <b>{max_value}</b> at longitude <b>{max_lon}</b> and latitude <b>{max_lat}</b></i>",
#     xref="paper", yref="paper",
#     x=0.5, y=1.2, showarrow=False,
#     font=dict(size=14),
#     align="center"
#     )

#     fig.add_annotation(
#     text=f"<i>Minimum value (in ppm): <b>{min_value}</b> at longitude <b>{min_lon}</b> and latitude <b>{min_lat}</b></i>",
#     xref="paper", yref="paper",
#     x=0.5, y=1.1, showarrow=False,
#     font=dict(size=14),
#     align="center"
#     )

#     # Add a title to the map
#     fig.update_layout(
#     title=f"<b>Stream Sediment samples showing {element} Values(ppm)</b>",
#     title_x=0.5,  # Center the title
#     title_y=0.95,  # Add some space from the top
#     margin=dict(t=120)  # Adjust margin to accommodate the annotations and title
#     )
# #     fig.show()
#     data = fig.to_dict()
#     layout = fig.to_dict()
#     return (data,layout, 'kriging_map')

#NEW KRIGING CODE - EXP:
def generate_kriging_map(df, element, toposheet_number, variogram_model='spherical'):
    # try printing the element, toposheet_number and variogram_model

    try:
         # Function body goes here
     # Filter the DataFrame by the specified toposheet number if provided
        if toposheet_number is not None:
            df = df[df['toposheet'] == toposheet_number]
            
    # Define grid resolution
        gridx = np.linspace(df['longitude'].min(), df['longitude'].max(), 100)
        gridy = np.linspace(df['latitude'].min(), df['latitude'].max(), 100)
        print("GRID X: \n", gridx)              
        print("GRID Y: \n", gridy)
        print("Longitude:", df['longitude'].values)
        print("Latitude:", df['latitude'].values)
        print("Element Values:", df[element].values)
        print("Variogram Model:", variogram_model)
    # Perform Ordinary Kriging
        print("Data head:", df.head())
        print("Data description:", df.describe())
        print("Longitude range:", df['longitude'].min(), df['longitude'].max())
        print("Latitude range:", df['latitude'].min(), df['latitude'].max())
        OK = OrdinaryKriging(df['longitude'], df['latitude'], df[element], variogram_model=variogram_model)
        z_interp, ss = OK.execute('grid', gridx, gridy)
        print("Z Interpolated:", z_interp)
        print("Semi-variance:", ss)
    # Create the contour plot
        contour = go.Contour(
            z=z_interp.data,  # 2D array of the heatmap values
            x=gridx,  # X coordinates corresponding to 'z_interp'
            y=gridy,  # Y coordinates corresponding to 'z_interp'
            colorscale='YlOrRd',  # Match the colormap
            showscale=True  # Show the color scale bar
        )
    
    # Create the scatter plot with hover annotations
        scatter = go.Scatter(
            x=df['longitude'],
            y=df['latitude'],
            mode='markers',
            marker=dict(
                color=df[element],
                colorscale='YlOrRd',  # Match the colormap
                showscale=False,  # We already have a color scale from the contour
                line=dict(color='black', width=1)  # Black border around the scatter points
            ),
            text=df[element],  # Text for hover annotations
            hoverinfo='text'  # Show only the text on hover
        )
        max_value = df[element].max()
        max_location = df[df[element] == max_value][['latitude', 'longitude']].iloc[0]
        max_lat, max_lon = max_location['latitude'], max_location['longitude']
            # Minimum value aur uske corresponding latitude, longitude find kar rahe hain
        min_value = df[element].min()
        min_location = df[df[element] == min_value][['latitude', 'longitude']].iloc[0]
        min_lat, min_lon = min_location['latitude'], min_location['longitude']
    
    # Create a figure and add the contour plot
        fig = go.Figure(data=[contour])
    
    # Add the scatter plot on top of the contour plot
#         fig.add_trace(scatter)
    
        # Update layout with title and labels
    #     fig.update_layout(
    #         title=f'Geochemical Kriging Map for {element}' + (f' (Toposheet {toposheet_number})' if toposheet_number else ''),
    #         xaxis_title='Longitude',
    #         yaxis_title='Latitude',
    #         coloraxis_colorbar=dict(title=f'{element} Concentration')
    #     )
        fig.add_annotation(
        text=f"<i>Maximum value (in ppm): <b>{max_value}</b> at longitude <b>{max_lon}</b> and latitude <b>{max_lat}</b></i>",
        xref="paper", yref="paper",
        x=0.5, y=1.2, showarrow=False,
        font=dict(size=14),
        align="center"
        )

        fig.add_annotation(
        text=f"<i>Minimum value (in ppm): <b>{min_value}</b> at longitude <b>{min_lon}</b> and latitude <b>{min_lat}</b></i>",
        xref="paper", yref="paper",
        x=0.5, y=1.1, showarrow=False,
        font=dict(size=14),
        align="center"
        )

    # Add a title to the map
        fig.update_layout(
        title=f"<b>Stream Sediment samples showing {element} Values(ppm)</b>",
        title_x=0.5,  # Center the title
        title_y=0.95,  # Add some space from the top
        margin=dict(t=120)  # Adjust margin to accommodate the annotations and title
        )
      
        data = fig.to_dict()
        layout = fig.to_dict()
        return (data,layout, 'kriging_map')
    except Exception as e:
        return 'Data for requested toposheet number is not available or updated. Please stay tuned for updates!'




def create_idw_map_from_query(query,df):
    t2 = extract_topo_no(query)
    e2 = extract_chemicals(query)
    threshold_percentile = 100   
    for toposheet_number in t2:
        for element in e2:
            max_value = df[element].max()
            max_location = df[df[element] == max_value][['latitude', 'longitude']].iloc[0]
            max_lat, max_lon = max_location['latitude'], max_location['longitude']
            # Minimum value aur uske corresponding latitude, longitude find kar rahe hain
            min_value = df[element].min()
            min_location = df[df[element] == min_value][['latitude', 'longitude']].iloc[0]
            min_lat, min_lon = min_location['latitude'], min_location['longitude']
            return generate_idw_map(df, element, toposheet_number, threshold_percentile) 
            # return generate_idw_map(df, element,max_value, max_location, max_lat, max_lon, min_value, min_location, min_lat, min_lon, toposheet_number, threshold_percentile) 


# In[181]:



# def generate_idw_map(df, element,max_value, max_location, max_lat, max_lon, min_value, min_location, min_lat, min_lon, toposheet_number, threshold_percentile):
#     # Filter the DataFrame by the specified toposheet number
#     gdf = df[df['toposheet'] == toposheet_number]

#     # Step 1: Determine Baseline
#     baseline = np.median(gdf[element])  # Using median as the baseline

#     # Step 2: Calculate Deviation
#     deviation = gdf[element] - baseline

#     # Step 3: Statistical Analysis
#     std_dev = np.std(deviation)
#     percentile_value = np.percentile(deviation, threshold_percentile)

#     # Step 4: Define Anomaly Threshold
#     anomaly_threshold = percentile_value  # Setting the specified percentile as the threshold

#     # Step 5: Identify Anomalies
#     anomalies = gdf[deviation > anomaly_threshold]

#     # Step 6: Anomaly Map
#     # Create grid coordinates for interpolation
#     grid_x, grid_y = np.mgrid[min(gdf['longitude']):max(gdf['longitude']):100j, min(gdf['latitude']):max(gdf['latitude']):100j]

#     # Interpolate using IDW
#     grid_z = griddata((gdf['longitude'], gdf['latitude']), deviation, (grid_x, grid_y), method='cubic')

#     # Plotting the IDW map
#     plt.figure(figsize=(10, 8))
#     plt.imshow(grid_z.T, extent=(min(gdf['longitude']), max(gdf['longitude']), min(gdf['latitude']), max(gdf['latitude'])), origin='lower')
#     plt.scatter(anomalies['longitude'], anomalies['latitude'], c='red')  # Anomalies marked in red
#     plt.colorbar(label='Deviation from Baseline')
#     plt.title(f'Geochemical IDW Map for {element} (Toposheet {toposheet_number})\n\nMaximum value (in ppm): {max_value} at longitude {max_lat} and latitude {max_lat}\nMinimum value (in ppm): {min_value} at longitude {min_lat} and latitude {min_lat}')

#     plt.xlabel('Longitude')
#     plt.ylabel('Latitude')
    
  
#     buffer = io.BytesIO()
#     plt.savefig(buffer, format='png')  # Save the plot to the buffer in PNG format
#     buffer.seek(0)
#     image_data = base64.b64encode(buffer.read()).decode('utf-8')  # Convert the image buffer to base64 string
#     output = {"image_data": image_data}
#     plt.close()  # Close the plot to prevent it from being displayed or printed
    
#     return output  # Return the base64 encoded image data as a dictionary

# OLD IDW CODE 
# def generate_idw_map(df, element, max_value, max_location, max_lat, max_lon, min_value, min_location, min_lat, min_lon, toposheet_number, threshold_percentile):
#     # Filter the DataFrame by the specified toposheet number
#     gdf = df[df['toposheet'] == toposheet_number]

#     # Determine Baseline
#     baseline = np.median(gdf[element])

#     # Calculate Deviation
#     deviation = gdf[element] - baseline

#     # Statistical Analysis
#     std_dev = np.std(deviation)
#     percentile_value = np.percentile(deviation, threshold_percentile)

#     # Define Anomaly Threshold
#     anomaly_threshold = percentile_value

#     # Identify Anomalies
#     anomalies = gdf[deviation > anomaly_threshold]

#     # Create grid coordinates for interpolation
#     grid_x, grid_y = np.mgrid[min(gdf['longitude']):max(gdf['longitude']):100j, min(gdf['latitude']):max(gdf['latitude']):100j]

#     # Interpolate using IDW
#     grid_z = griddata((gdf['longitude'], gdf['latitude']), deviation, (grid_x, grid_y), method='cubic')

#     # Create the contour plot
#     contour = go.Contour(
#         z=grid_z.T,
#         x=np.linspace(min(gdf['longitude']), max(gdf['longitude']), 100),
#         y=np.linspace(min(gdf['latitude']), max(gdf['latitude']), 100),
#         colorscale='Viridis',
#         colorbar=dict(title='Deviation from Baseline')
#     )

#     # Create the scatter plot for anomalies
#     scatter = go.Scatter(
#         x=anomalies['longitude'],
#         y=anomalies['latitude'],
#         mode='markers',
#         marker=dict(color='red', size=5),
#         name='Anomalies'
#     )

#     # Define the layout
#     layout = go.Layout(
#         title=f'Geochemical IDW Map for {element} (Toposheet {toposheet_number})<br>'
#               f'Maximum value (in ppm): {max_value} at longitude {max_lon} and latitude {max_lat}<br>'
#               f'Minimum value (in ppm): {min_value} at longitude {min_lon} and latitude {min_lat}',
#         xaxis=dict(title='Longitude'),
#         yaxis=dict(title='Latitude')
#     )
    
#     # Create the figure and plot it
#     fig = go.Figure(data=[contour, scatter], layout=layout)
#     data = fig.to_dict()
#     layout = fig.to_dict()
#     return (data, layout, 'idw_map')

#NEW IDW CODE - EXP:
def generate_idw_map(df, element, toposheet_number, threshold_percentile):
    # Filter the DataFrame by the specified toposheet number
    gdf = df[df['toposheet'] == toposheet_number]
    if (gdf[element] == 0.00).all():
        print('Data for the requested toposheet number is not available or updated. Please stay tuned for updates!')
    else:
        # Determine Baseline
        baseline = np.median(gdf[element])

        # Calculate Deviation
        deviation = gdf[element] - baseline

        # Statistical Analysis
        std_dev = np.std(deviation)
        percentile_value = np.percentile(deviation, threshold_percentile)

        # Define Anomaly Threshold
        anomaly_threshold = percentile_value

        # Identify Anomalies
        anomalies = gdf[deviation > anomaly_threshold]
        max_value = gdf[element].max()
        max_location = gdf[gdf[element] == max_value][['latitude', 'longitude']].iloc[0]
        max_lat, max_lon = max_location['latitude'], max_location['longitude']
        # Minimum value aur uske corresponding latitude, longitude find kar rahe hain
        min_value = gdf[element].min()
        min_location = gdf[gdf[element] == min_value][['latitude', 'longitude']].iloc[0]
        min_lat, min_lon = min_location['latitude'], min_location['longitude']

        # Create grid coordinates for interpolation
        grid_x, grid_y = np.mgrid[min(gdf['longitude']):max(gdf['longitude']):100j, min(gdf['latitude']):max(gdf['latitude']):100j]

        # Interpolate using IDW
        grid_z = griddata((gdf['longitude'], gdf['latitude']), deviation, (grid_x, grid_y), method='cubic')

        # Create the contour plot
        contour = go.Contour(
            z=grid_z.T,
            x=np.linspace(min(gdf['longitude']), max(gdf['longitude']), 100),
            y=np.linspace(min(gdf['latitude']), max(gdf['latitude']), 100),
            colorscale='Viridis',
            colorbar=dict(title='Deviation from Baseline')
        )

        # Create the scatter plot for anomalies
        scatter = go.Scatter(
            x=anomalies['longitude'],
            y=anomalies['latitude'],
            mode='markers',
            marker=dict(color='red', size=5),
            name='Anomalies'
        )        

        # Add annotations for maximum and minimum values
        annotations = [
            dict(
                text=f"<i>Maximum value (in ppm): <b>{max_value}</b> at longitude <b>{max_lon}</b> and latitude <b>{max_lat}</b></i>",
                xref="paper", yref="paper",
                x=0.5, y=1.2, showarrow=False,
                font=dict(size=14),
                align="center"
            ),
            dict(
                text=f"<i>Minimum value (in ppm): <b>{min_value}</b> at longitude <b>{min_lon}</b> and latitude <b>{min_lat}</b></i>",
                xref="paper", yref="paper",
                x=0.5, y=1.1, showarrow=False,
                font=dict(size=14),
                align="center"
            )
        ]
        
        # Define the layout with annotations
        layout = go.Layout(
            annotations=annotations,
            title=f"<b>Stream Sediment samples showing {element} Values(ppm)</b>",
            title_x=0.5,
            title_y=0.95,
            margin=dict(t=120) 
        )

        # Create the figure and plot it
        fig = go.Figure(data=[contour, scatter], layout=layout)
        data = fig.to_dict()
        layout = fig.to_dict()
        return (data, layout, 'idw_map')
        

# Example usage:
# generate_idw_map(df, 'element_name', toposheet_number, threshold_percentile)


def find_max_values(query, df):
    toposheet_numbers = extract_topo_no(query)
    elements = extract_chemicals(query)
    for toposheet_number in toposheet_numbers:
        for element in elements:
            max_value = df[element].max()
            max_location = df[df[element] == max_value][['latitude', 'longitude']].iloc[0]
            max_lat, max_lon = max_location['latitude'], max_location['longitude']
            max_value_result = f"For the toposheet {toposheet_number}, the element {element} has maximum PPM value {max_value} at latitude {max_lat} and longitude {max_lon}."
            return max_value_result


# In[183]:


def find_min_values(query, df):
#     print("[INFO:] Finding the min values")
    toposheet_numbers = extract_topo_no(query)
    elements = extract_chemicals(query)
    for toposheet_number in toposheet_numbers:
        for element in elements:
            min_value = df[element].min()
            min_location = df[df[element] == min_value][['latitude', 'longitude']].iloc[0]
            min_lat, min_lon = min_location['latitude'], min_location['longitude']
    
            # Results ko sentence mein display kar rahe hain
            min_value_result = f"For the toposheet {toposheet_number}, the element {element} has minimum PPM value {min_value} at latitude {min_lat} and longitude {min_lon}."
            return min_value_result


# In[184]:


def find_both_min_max(query, df):
    toposheet_numbers = extract_topo_no(query)
    elements = extract_chemicals(query)
    for toposheet_number in toposheet_numbers:
        for element in elements:
            max_value = df[element].max()
            max_location = df[df[element] == max_value][['latitude', 'longitude']].iloc[0]
            max_lat, max_lon = max_location['latitude'], max_location['longitude']
            # Minimum value aur uske corresponding latitude, longitude find kar rahe hain
            min_value = df[element].min()
            min_location = df[df[element] == min_value][['latitude', 'longitude']].iloc[0]
            min_lat, min_lon = min_location['latitude'], min_location['longitude']

            # Results ko sentence mein display kar rahe hain
            min_max_value_result = f"For the toposheet {toposheet_number}, the element {element} has maximum PPM value {max_value} at latitude {max_lat} and longitude {max_lon}, and has minimum PPM value {min_value} at latitude {min_lat} and longitude {min_lon}."
            return min_max_value_result


# In[185]:


def split_query_smartly(query):
    element_names = ['silicon dioxide', 'aluminum oxide', 'iron(III) oxide', 'titanium dioxide', 'calcium oxide', 'magnesium oxide', 'manganese(II) oxide', 'sodium oxide', 'potassium oxide', 'phosphorus pentoxide', 'loss on ignition', 'barium', 'gallium', 'scandium', 'vanadium', 'thorium', 'lead', 'nickel', 'cobalt', 'rubidium', 'strontium', 'yttrium', 'zirconium', 'niobium', 'chromium', 'copper', 'zinc', 'gold', 'lithium', 'cesium', 'arsenic', 'antimony', 'bismuth', 'selenium', 'silver', 'beryllium', 'germanium', 'molybdenum', 'tin', 'lanthanum', 'cerium', 'praseodymium', 'neodymium', 'samarium', 'europium', 'terbium', 'gadolinium', 'dysprosium', 'holmium', 'erbium', 'thulium', 'ytterbium', 'lutetium', 'hafnium', 'tantalum', 'tungsten', 'uranium', 'platinum', 'palladium', 'indium', 'fluorine', 'tellurium', 'thallium', 'mercury', 'cadmium']
    # Load spaCy English model
    nlp = spacy.load("en_core_web_sm")
    
    # Define split words and phrases
    split_words = ["maximum and minimum", "longitude and latitude"]
    additional_split_words = ["also", "display", "create", "produce", "tell", "describe","show"]
    
    # Process the query using spaCy
    doc = nlp(query)
    
    # Initialize variables
    subqueries = []
    subquery = ""
    split_flag = False  # Flag to determine if splitting is allowed
    
    # Iterate over tokens in the query
    for token in doc:
        # Check if token is a split word or phrase
        if any(split_word in token.text.lower() for split_word in split_words):
            split_flag = False
        elif token.text.strip() == "." or token.text.strip() == ". ":
            # Split when encountering a full stop or full stop followed by a space
            if subquery.strip():
                subqueries.append(subquery.strip())
            subquery = ""
            split_flag = False
        elif token.text.strip() in additional_split_words:
            # Split when encountering additional split words
            if subquery.strip():
                subqueries.append(subquery.strip())
            subquery = ""
            split_flag = False
        elif token.text.strip() == "and" and split_flag:
            # Do not split when "and" is encountered between elements
            subquery += token.text_with_ws
        elif token.text.strip() == "and" and subquery.lower().strip() in element_names:
            # Do not split when "and" is encountered after an element name
            subquery += token.text_with_ws
        elif token.text.strip().lower() not in additional_split_words:
            # Append token to subquery if it's not in the additional split words list
            subquery += token.text_with_ws
            split_flag = True  # Set flag to allow splitting after this token
    
    # Append the final subquery if it's not empty and contains more than one word
    if subquery.strip() and len(subquery.split()) > 1:
        subqueries.append(subquery.strip())
    
    return subqueries



def process_subqueries(subqueries):
#     combined_output = []  # Initialize a list to store combined output
    df = Nagpur_gdf
    try: 
        for subquery in subqueries:
            print(f"Processing subquery: {subquery}")
            if ('maximum' in subquery.lower() and 'minimum' in subquery.lower()) or ('max' in subquery.lower() and 'min' in subquery.lower()):
                print("Calling the function for min and max")
                output = find_both_min_max(subquery, df=Nagpur_gdf) 
            elif ('maximum' in subquery.lower()) or ('max' in subquery.lower()):
                print('Calling Max Fun')
                output = find_max_values(subquery,df)
            elif ('minimum' in subquery.lower()) or ('min' in subquery.lower()):
                print('Calling Min Fun')
                output = find_min_values(subquery,df)      
                
            # Check if the subquery mentions 'idw interpolation map'
            elif 'idw' in subquery.lower() or 'inverse distance weighted map' in subquery.lower():
                print("Calling the IDW function")
                output = create_idw_map_from_query(subquery, df)           
                
            elif 'idw' in subquery.lower() and "kriging" in subquery.lower():
                print("Calling both kriging and IDW function")
                output_idw = create_idw_map_from_query(subquery, df)
                output_kriging = create_kriging_map_from_query(subquery, df)
                output = (output_idw, output_kriging)  # Store both outputs in a tuple
                
            elif 'kriging' in subquery.lower():
                print("Calling the kriging function")
                output = create_kriging_map_from_query(subquery, df)
            else:
                apology_message = "I'm only able to provide information related to Nagpur data. Please enter a valid query about Nagpur toposheet data. Thank you for your understanding and patience."
                print(apology_message)
        return output 
    except Exception as e:
        print("Error:", e)
        return "There was problem processing your query, please try again and make sure the query is valid on Nagpur toposheet data. Thank you for your understanding and patience."
# Example usage:
# SubQueries = [
#     'the maximum and minimum longitude and latitude values for 55K14 for gold',
#     'a kriging map for the gold for 55K14'
# ]

# combined_output = process_subqueries(SubQueries)
# print("Combined Output:", combined_output)


# In[200]:


def generate_geochemistry_response(query):
    corrected_sentence = correct_typos(query)
    Subqueries = split_query_smartly(corrected_sentence)
    response = process_subqueries(Subqueries)
    if response == None:
        response = "Sorry, I am unable to respond to this query. I am currently equipped to provide information on Nagpur Geochemistry Toposheet data and can handle one query at a time. Thank you for your understanding."
    data_type = "text"
    if type(response) == tuple and response[2]=='idw_map':
        data_type = "idw_map"
    elif type(response) == tuple and response[2]=='kriging_map':
        data_type = "kriging_map"
    return response, data_type


if __name__ == "__main__":
    generate_geochemistry_response(query="Create a kriging map for silver for the toposheet number 55K14")


Processing subquery: a kriging map for silver for the toposheet number 55K14
Calling the kriging function
GRID X: 
 [78.75965    78.76199374 78.76433747 78.76668121 78.76902495 78.77136869
 78.77371242 78.77605616 78.7783999  78.78074364 78.78308737 78.78543111
 78.78777485 78.79011859 78.79246232 78.79480606 78.7971498  78.79949354
 78.80183727 78.80418101 78.80652475 78.80886848 78.81121222 78.81355596
 78.8158997  78.81824343 78.82058717 78.82293091 78.82527465 78.82761838
 78.82996212 78.83230586 78.8346496  78.83699333 78.83933707 78.84168081
 78.84402455 78.84636828 78.84871202 78.85105576 78.85339949 78.85574323
 78.85808697 78.86043071 78.86277444 78.86511818 78.86746192 78.86980566
 78.87214939 78.87449313 78.87683687 78.87918061 78.88152434 78.88386808
 78.88621182 78.88855556 78.89089929 78.89324303 78.89558677 78.89793051
 78.90027424 78.90261798 78.90496172 78.90730545 78.90964919 78.91199293
 78.91433667 78.9166804  78.91902414 78.92136788 78.92371162 78.92605535
 78.9283

Z Interpolated: [[132.99999999999946 101.56554952926604 100.57159942808539 ...
  75.34849292199921 74.76638492540353 74.27314555630005]
 [98.3087420556416 98.59352674295545 98.04974938625871 ...
  76.13185357336192 75.58553498312321 75.16611257057812]
 [93.40869741654492 94.04963188773651 94.04696415200642 ...
  77.06884544307252 76.57914403547805 76.15977792161556]
 ...
 [86.47933323659417 84.27117903573365 81.1280336301513 ...
  104.35802495324043 104.30361107373844 103.34758638459247]
 [87.01499722284333 84.30453275470947 80.54677826550532 ...
  103.63121941743597 104.13781673364224 103.6719589450737]
 [87.90447672004684 84.00990239686125 79.80794468983214 ...
  102.11186718688992 103.10731353075695 127.99999999999929]]
Semi-variance: [[-1.2558936559817853e-11 1697.271237962871 1759.255159406948 ...
  1750.5280693091074 1684.952266962281 1632.559061520683]
 [1695.3044530288307 1718.8865507927699 1763.4609116030197 ...
  1756.490579546652 1711.9848370645677 1697.824142290517]
 [1753.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import scipy
import spacy
import plotly
import pykrige

print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("SciPy version:", scipy.__version__)
print("spaCy version:", spacy.__version__)
print("Plotly version:", plotly.__version__)

# PyKrige doesn't have a version attribute directly accessible like other libraries,
# You might need to check it using pip or setup.py if it's a custom install.
# You can try this, but it might not work if the attribute isn't set:
try:
    import pykrige
    print("PyKrige version:", pykrige.__version__)
except AttributeError:
    print("PyKrige version not directly accessible.")

Pandas version: 1.5.3
NumPy version: 1.24.3
Matplotlib version: 3.7.1
SciPy version: 1.10.1
spaCy version: 3.7.4
Plotly version: 5.9.0
PyKrige version: 1.7.1
